In [9]:
# imports
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.ccny.cuny.edu/registrar/fall"
default_year = ',  2021' # necessary to fill in blank years

In [ ]:
html_doc = requests.get(url).text
soup = BeautifulSoup(html_doc)
# print(soup.prettify())
table = soup.find_all('table')[0]
# print(table)

rows = table.find_all('tr')
# print(rows)

data = {
    "date": [],
    "dow": [], # day of week
    "text": []
}

def clean_date(s: str) -> str:
    # takes the start of a range of dates and adds the default year if necessary
    start = s.strip().split('-')[0]
    if ',' not in start:
        start += default_year
    return pd.to_datetime(start).date()

def clean_dow(s: str) -> str:
    # keeps the first word only (day of week), used for ranges of days
    return s.strip().replace("\n", " ").replace("\t", " ").split()[0]

def clean_text(s: str) -> str:
    # removes extra spaces, newlines, tabs
    return " ".join(s.strip().split())

for tr in rows:
    tds = tr.find_all('td')
    if tds:
        data["date"].append(clean_date(tds[0].get_text()))
        data["dow"].append(clean_dow(tds[1].get_text()))
        data["text"].append(clean_text(tds[2].get_text()))

df = pd.DataFrame(data)
df["date"] = pd.to_datetime(df["date"]).dt.date
df = df.set_index("date").sort_index()

df

,date,dow,text
0,2021-08-01,Sunday,Application for degree for January and Februar...
1,2021-08-18,Wednesday,Last day to apply for Study Abroad
2,2021-08-24,Tuesday,Last day of Registration; Last day to file ePe...
3,2021-08-25,Wednesday,Start of Fall Term; Classes begin; Initial Reg...
4,2021-08-25,Wednesday,Change of program period; late fees apply
5,2021-08-26,Thursday,Last day for Independent Study
6,2021-08-28,Saturday,First day of Saturday Classes
7,2021-08-31,Tuesday,Last day to add a class to an existing enrollm...
8,2021-09-01,Wednesday,Verification of Enrollment rosters available t...
9,2021-09-03,Friday,No classes scheduled
